In [1]:
import random
import regex
import re
import sympy
import sys
sys.path.append("..")
from parser import strip_string
# from latexb
def extract_answer(pred_str, data_name, use_last_number=True):
    pred_str = pred_str.replace("\u043a\u0438", "")

    if "final answer is $" in pred_str and "$. I hope" in pred_str:
        # minerva_math
        tmp = pred_str.split("final answer is $", 1)[1]
        pred = tmp.split("$. I hope", 1)[0].strip()
    elif "boxed" in pred_str:
        ans = pred_str.split("boxed")[-1]
        if len(ans) == 0:
            return ""
        elif ans[0] == "{":
            stack = 1
            a = ""
            for c in ans[1:]:
                if c == "{":
                    stack += 1
                    a += c
                elif c == "}":
                    stack -= 1
                    if stack == 0:
                        break
                    a += c
                else:
                    a += c
        else:
            a = ans.split("$")[0].strip()
        pred = a
    elif "he answer is" in pred_str:
        pred = pred_str.split("he answer is")[-1].strip()
    elif "final answer is" in pred_str:
        pred = pred_str.split("final answer is")[-1].strip()
    elif "答案是" in pred_str:
        # Handle Chinese few-shot multiple choice problem answer extraction
        pred = pred_str.split("答案是")[1].strip().split("\n\n")[0].strip()
    else:  # use the last number
        if use_last_number:
            pattern = "-?\d*\.?\d+"
            pred = re.findall(pattern, pred_str.replace(",", ""))
            if len(pred) >= 1:
                pred = pred[-1]
            else:
                pred = ""
        else:
            pred = ""

    # choice answer
    if (
        data_name in ["sat_math", "aqua"]
        or "mmlu" in data_name
    ):
        tmp = re.findall(r"\b(A|B|C|D|E)\b", pred.upper())
        if tmp:
            pred = tmp[-1]
        else:
            pred = pred.strip().strip(".")

    # multiple line
    # pred = pred.split("\n")[0]
    pred = re.sub(r"\n\s*", "", pred)
    if pred != "" and pred[0] == ":":
        pred = pred[1:]
    if pred != "" and pred[-1] == ".":
        pred = pred[:-1]
    if pred != "" and pred[-1] == "/":
        pred = pred[:-1]
    pred = strip_string(pred, skip_unit=data_name in ["carp_en", "minerva_math"])
    return pred

In [9]:
def extract_answer(pred_str, data_name, use_last_number=True):
    pred_str = pred_str.replace("\u043a\u0438", "")

    if "final answer is $" in pred_str and "$. I hope" in pred_str:
        # minerva_math
        tmp = pred_str.split("final answer is $", 1)[1]
        pred = tmp.split("$. I hope", 1)[0].strip()
    elif "boxed" in pred_str:
        ans = pred_str.split("boxed")[-1]
        if len(ans) == 0:
            return ""
        elif ans[0] == "{":
            stack = 1
            a = ""
            for c in ans[1:]:
                if c == "{":
                    stack += 1
                    a += c
                elif c == "}":
                    stack -= 1
                    if stack == 0:
                        break
                    a += c
                else:
                    a += c
        else:
            a = ans.split("$")[0].strip()
        pred = a
    elif "he answer is" in pred_str:
        pred = pred_str.split("he answer is")[-1].strip()
    elif "final answer is" in pred_str:
        pred = pred_str.split("final answer is")[-1].strip()
    elif "答案是" in pred_str:
        # Handle Chinese few-shot multiple choice problem answer extraction
        pred = pred_str.split("答案是")[1].strip().split("\n\n")[0].strip()
    else:  # use the last number
        if use_last_number:
            pattern = "-?\d*\.?\d+"
            pred = re.findall(pattern, pred_str.replace(",", ""))
            if len(pred) >= 1:
                pred = pred[-1]
            else:
                pred = ""
        else:
            pred = ""

    # choice answer
    if (
        data_name in ["sat_math", "aqua"]
        or "mmlu" in data_name
    ):
        tmp = re.findall(r"\b(A|B|C|D|E)\b", pred.upper())
        if tmp:
            pred = tmp[-1]
        else:
            pred = pred.strip().strip(".")

    # multiple line
    # pred = pred.split("\n")[0]
    pred = re.sub(r"\n\s*", "", pred)
    if pred != "" and pred[0] == ":":
        pred = pred[1:]
    if pred != "" and pred[-1] == ".":
        pred = pred[:-1]
    if pred != "" and pred[-1] == "/":
        pred = pred[:-1]
    pred = strip_string(pred, skip_unit=data_name in ["carp_en", "minerva_math"])

    return pred

In [5]:
pred_str = 'The final answer is:$\\boxed{even}$'
pred_str = "We multiply both sides of the equation by $10x$ to clear the fractions, leaving us with $2x^2 + 2 = 5x$. Rearranging the terms, we have $2x^2 - 5x + 2 = 0$. We can now solve for $x$ by factoring: $(2x - 1)(x - 2) = 0$. We could also use the quadratic formula:  $$x = \\frac{5 \\pm \\sqrt{(-5)^2 - 4(2)(2)}}{4}.$$Either way, we find that $x = 1/2$ or $x = 2$. Since we want the largest value of $x$, our answer is $\\boxed 2$."
pred_str = """## Step 1: Convert the given numbers from base 9 to base 10 for easier computation.
First, we need to convert $58_9$ and $18_9$ into base 10. To do this, we'll use the formula for converting a number from any base to base 10: $d_n\times b^n + \cdots + d_2\times b^2 + d_1\times b^1 + d_0\times b^0$, where $b$ is the base of the original number and $d_i$ are its digits. For $58_9$, it becomes $5\times 9^1 + 8\times 9^0 = 45 + 8 = 53$. For $18_9$, it becomes $1\times 9^1 + 8\times 9^0 = 9 + 8 = 17$.

## Step 2: Perform subtraction in base 10.
Now that both numbers are in base 10, we can perform the subtraction: $53 - 17$.

## Step 3: Calculate the result of the subtraction.
Carrying out the subtraction gives us $36$.

## Step 4: Convert the result back to base 9.
To express our final result in base 9, we need to convert $36_{10}$ back into base 9. We will divide 36 by 9 repeatedly and keep track of the remainders.

## Step 5: Divide 36 by 9 and find the remainder.
Dividing 36 by 9 gives a quotient of 4 with no remainder.

## Step 6: Continue dividing the quotient by 9 until reaching zero.
Next, we divide 4 by 9: 4 divided by 9 gives a quotient of 0 with a remainder of 4.

## Step 7: Construct the base 9 representation using the remainders obtained in reverse order.
The base 9 representation of $36_{10}$ is therefore $44_9$.

## Step 8: The resulting expression should be expressed as requested in the problem.

## Step 9: Write down the final solution.

## Step 10: Double-check the work.

## Step 11: Determine if further calculations or checks are needed.

## Step 12: State the final answer.

## Step 13: Provide the expression in base 9.

## Step 14: Conclude.

## Step 15: Present the final answer.

## Step 16: Solve for 58_9 - 18_9 in base 9.

## Step 17: Subtract 18_9 from 58_9 in base 9.

## Step 18: Perform the calculation of 58_9 - 18_9 in base 9.

## Step 19: Convert 58_9 to base 10 first.



The final answer is:$\\boxed{44_9}$"""
data_name = "math"
pred = extract_answer(pred_str, data_name)
print(pred)

44_9
